In [ ]:
import pandas as pd

# Read the first CSV file
df1 = pd.read_csv('2020_new.csv')

# Read the second CSV file
df2 = pd.read_csv('2021_new.csv')

df2.columns = df1.columns


df = pd.concat([df1, df2], ignore_index = True)

df = df.drop(df.columns[:5], axis=1)
df = df.drop(df.columns[4], axis=1)
df = df.drop(df.columns[2], axis=1)
df = df.drop(df.columns[5], axis=1)
print(df.columns)

mapping = {val: i for i, val in enumerate(df['D'].unique())}
print("Grade Mapping:")
for grade, number in mapping.items():
    print(f"{grade}: {number}")
# # use the map() method to map the strings to numerical values
df['D'] = df['D'].map(mapping)

df.head()

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# define the input and output dimensions of the model
input_dim = len(df.columns) - 1
output_dim = 8  # 7 unique classes + no class category

# # convert the categorical output feature into integer labels
# label_mapping = {'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3, 'class5': 4, 'class6': 5, 'class7': 6, 'no class': 7}
# df['D'] = df['D'].map(label_mapping)


# split the dataset into training, validation, and testing sets
X_train, X_val, y_train, y_val = train_test_split(df.iloc[:, :2].join(df.iloc[:, -3:]), df.iloc[:, 2], test_size=0.2, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0)

# convert the data into PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:


# define the neural network model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,64)
        self.fc5 = nn.Linear(64,output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.softmax(x)
        return x




In [ ]:
print(y_train)

In [ ]:

# instantiate the model and the optimizer
net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.001)

num_epochs = 500
batch_size = 5
criterion = nn.CrossEntropyLoss()
train_losses = []
valid_losses = []
best_accuracy = 0.0  # variable to keep track of the best accuracy
for epoch in range(num_epochs):
    epoch_loss = 0
    optimizer.zero_grad()
    output = net(X_train)
    loss = criterion(output,y_train)
    loss.backward()
    optimizer.step()
    
    # add the loss value for each iteration to the epoch_loss variable
    epoch_loss += loss.item()

    # calculate the average loss for the epoch and append it to the train_losses list
    epoch_loss /= len(X_train)
    train_losses.append(epoch_loss)
    print(f"Epoch: {epoch}, Epoch_loss: {epoch_loss}")

    # evaluate the model on the validation set
    net.eval()
    with torch.no_grad():
        output = net(X_val)
        predicted = torch.argmax(output, axis=1)
        accuracy = torch.mean((predicted == y_val).float())
        print(f'Validation accuracy: {accuracy}')
        valid_losses.append(accuracy)
        # save the model weights with the highest validation accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(net.state_dict(), 'best_weights.pt')
  
    net.train()


In [ ]:
net.load_state_dict(torch.load('best_weights.pt'))
# evaluate the model on the testing set
net.eval()
with torch.no_grad():
    output = net(X_test)
    predicted = torch.argmax(output, axis=1)
    print(predicted)
    print(y_test)
    accuracy = torch.mean((predicted == y_test).float())
    print(f'Testing accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss vs. Epoch")
plt.show()

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)